In [1]:
import pandas as pd

train_images_path = "/home/docker_current/datasets/train"
df_train = pd.read_csv("/home/docker_current/datasets/train.csv")
df_train['text_features'] = None
df_train['image_features'] = None
df_train['pred_id'] = 0
df_train.head(3)

,id,description,object_img,text_features,image_features,pred_id
0,520,Фотография. Г. Пермь. Здание горисполкома. ПО...,799,None,None,0
1,1817,Фотонегатив пленочный. Труппа театра «У моста»...,854,None,None,0
2,188,"Изразец гладкий расписной ""пермский""- карниз А...",1794,None,None,0


In [22]:
'''Загрузка multilangual-CLIP предиктора'''
from MLCLIP import MLCLIPPredictor
predictor = MLCLIPPredictor()

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
'''Загрузка ruCLIP предиктора'''
# from ruCLIP import RUCLIPPredictor
# predictor = RUCLIPPredictor()

In [23]:
from PIL import Image

ind = 1
text = df_train['description'][ind]
sample_image_path = "/home/docker_current/datasets/train/" + str(df_train['object_img'][ind]) + ".png"
image = Image.open(sample_image_path)

text_features, image_features = predictor.inference(text, image)
image_features.shape, text_features.shape

((1, 768), (1, 768))

In [24]:
import torch
cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)

cos(torch.Tensor(text_features), torch.Tensor(image_features))

tensor([0.2799])

In [ ]:
# !pip install sentencepiece
# !pip install transformers[sentencepiece]

In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import numpy as np
import torch
import open_clip
from open_clip import tokenizer
import os
from tqdm import tqdm

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ru-en", use_fast=False)
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ru-en")

from transformers import pipeline

model_checkpoint = "Helsinki-NLP/opus-mt-ru-en"
translator = pipeline("translation", model=model_checkpoint, device=torch.device("cuda:0"))
translator("Как у тебя дела?")

Downloading:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/784k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/293M [00:00<?, ?B/s]

TypeError: '<' not supported between instances of 'torch.device' and 'int'

In [ ]:
model, _, preprocess = open_clip.create_model_and_transforms(
    'ViT-H-14', 
    pretrained='laion2b_s32b_b79k', 
    device=torch.device("cuda:0"))
model.eval();